In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import calendar
import re


# 기본 설정
options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://icare.seoul.go.kr"
list_url = f"{base_url}/icare/user/kidsCafe/BD_selectKidsCafeList.do"
today = datetime.today()
results = []
total_pages = 16

driver.get(list_url)

In [ ]:
results = []
current_page = 1

def click_jsMovePage(n):
    try:
        btn = driver.find_element(By.XPATH, f'//a[contains(@onclick, "jsMovePage({n});")]')
        driver.execute_script("arguments[0].click();", btn)
        print(f"{n}페이지 이동")
        time.sleep(2)
        return True
    except:
        return False

while True:
    print(f"현재 수집 중인 페이지: {current_page}")
    
    cafes = driver.find_elements(By.XPATH, '//*[@id="searchForm"]/div[2]/div')

    for one in cafes:
        try:
            name = one.find_element(By.XPATH, './/h5').text
            if '시립 옴팡점' in name:
                print(f"'{name}'은 건너뜀")
                continue

            capacity = one.find_element(By.XPATH, './/dl/dd').text.split()
            person_key = capacity[0]
            people_key = capacity[3]
            person = int(capacity[1])
            people = int(capacity[4])

            link = one.find_element(By.XPATH, './/a[5]')
            driver.execute_script("arguments[0].click();", link)
            time.sleep(2)

            month = driver.find_element(By.XPATH, '//*[@id="calForm"]/div[1]/div[1]/div[1]/span')
            if str(month.text) == '2025. 03':
                driver.find_element(By.XPATH, '//*[@id="calForm"]/div[1]/div[1]/div[1]/button[2]').send_keys(Keys.ENTER)
                time.sleep(2)

            day_xpath = '//*[@id="calendar"]/tbody/tr/td'
            num_days = len(driver.find_elements(By.XPATH, day_xpath))

            for i in range(num_days):
                days = driver.find_elements(By.XPATH, day_xpath)
                day = days[i]

                try:
                    title = driver.execute_script("return arguments[0].getAttribute('title');", day)
                    td_class = driver.execute_script("return arguments[0].getAttribute('class')", day)

                    if title and '예약가능' in title and (not td_class or 'disable' not in td_class):
                        reservation_status = title.split()[1]
                        date = '4월' + title.split()[0]

                        day.click()
                        try:
                            WebDriverWait(driver, 5).until(
                                EC.presence_of_element_located((By.XPATH, '//*[@id="calendar"]/tbody/tr/td/div'))
                            )
                        except:
                            print(f"[시간대 로딩 실패] {name} - {date} → 패스")
                            continue

                        time_blocks = day.find_elements(By.XPATH, './/div/p')
                        for times in time_blocks:
                            try:
                                raw = driver.execute_script("return arguments[0].innerText", times)
                                lines = [line.strip() for line in raw.splitlines() if line.strip()]
                                print("lines:", lines)

                                if not lines or all("운영일이 아닙니다" in l for l in lines):
                                    print(f"[운영일 아님] {name} - {date} → 저장 안 함")
                                    continue

                                r = 0
                                while r + 2 < len(lines):
                                    session = lines[r]
                                    numbers = lines[r + 1]
                                    count_str = lines[r + 2]

                                    if '자체휴회' in count_str:
                                        results.append({
                                            'name': name,
                                            'date': date,
                                            'reservation_status': reservation_status,
                                            'capacity_person': person,
                                            'capacity_people': people,
                                            'session': session,
                                            'numbers_status': numbers,
                                            'reservated_num': '자체휴회'
                                        })
                                        r += 3
                                        continue

                                    try:
                                        count = int(count_str)
                                    except ValueError:
                                        r += 3
                                        continue

                                    if numbers == person_key:
                                        real_res = person - count
                                    elif numbers == people_key:
                                        real_res = people - count
                                    else:
                                        real_res = person - count

                                    results.append({
                                        'name': name,
                                        'date': date,
                                        'reservation_status': reservation_status,
                                        'capacity_person': person,
                                        'capacity_people': people,
                                        'session': session,
                                        'numbers_status': numbers,
                                        'reservated_num': real_res
                                    })

                                    r += 3
                            except Exception as e:
                                print(f"[시간대 처리 오류] {e}")
                except Exception as e:
                    print(f"[날짜 처리 오류] {e}")

            driver.back()
            time.sleep(2)

        except Exception as e:
            print(f"[카페 처리 오류] {e}")

    next_page = current_page + 1
    if next_page % 10 == 1:
        try:
            print("다음 페이지 그룹 이동")
            next_group_btn = driver.find_element(By.XPATH, '//li[@class="next_btn"]/a')
            driver.execute_script("arguments[0].click();", next_group_btn)
            time.sleep(2)
            current_page = next_page
        except:
            print("다음 페이지 그룹 없음. 종료.")
            break
    else:
        success = click_jsMovePage(next_page)
        if success:
            current_page = next_page
        else:
            print("다음 페이지 없음. 종료.")
            break

# 오늘 기준 정보
today = datetime.today()
this_month = 4  # 지금은 4월 기준이니까 고정

# 날짜 숫자 추출 함수
def extract_day(date_str):
    try:
        # '4월5' → '4', '5' / '4월12일' → '4', '12'
        nums = re.findall(r'\d+', str(date_str))
        if not nums:
            return None
        for i in range(len(nums)-1):
            if nums[i] == '4':
                return int(nums[i+1])  # '4', '5' → 5
        # 혹시 '4월5' 자체가 '45'로 붙어 있는 경우 → 뒷자리만 따오기
        joined = nums[0]
        if joined.startswith('4') and len(joined) >= 2:
            return int(joined[1:])
        return None
    except Exception as e:
        print(f"[ERROR in extract_day()] date_str={date_str}, error={e}")
        return None

# 날짜가 오늘 이후이면서 유효한지 체크
def is_future_or_today(date_str):
    day = extract_day(date_str)
    if not day:
        print(f"[날짜 파싱 실패] '{date_str}'")
        return False

    max_day = calendar.monthrange(today.year, this_month)[1]
    if not (1 <= day <= max_day):
        print(f"[유효하지 않은 날짜] '{date_str}' (day={day})")
        return False

    date_obj = datetime(today.year, this_month, day)
    if date_obj < today:
        print(f"[과거일자 스킵] '{date_str}' → {date_obj.strftime('%Y-%m-%d')}")
        return False

    print(f"[유효일자] '{date_str}' → {date_obj.strftime('%Y-%m-%d')}")
    return True

# 수집 결과 데이터프레임 생성
new_df = pd.DataFrame(results)

# 기존 CSV 불러오기 or 새로 저장
try:
    old_df = pd.read_csv("./예약현황/서울형_키즈카페_예약현황.csv")
except FileNotFoundError:
    print("기존 파일 없음 → 전체 새로 저장")
    new_df.to_csv("./예약현황/서울형_키즈카페_예약현황.csv", index=False, encoding='utf-8')
else:
    # 문자열 비교 컬럼 전처리
    for col in ['name', 'date', 'session', 'numbers_status']:
        old_df[col] = old_df[col].astype(str).str.replace("일", "").str.strip()
        new_df[col] = new_df[col].astype(str).str.replace("일", "").str.strip()

    updated_rows = []

    # 행별 비교 및 추가/업데이트 판단
    for idx, new_row in new_df.iterrows():
        if not is_future_or_today(new_row['date']):
            continue

        mask = (
            (old_df['name'] == new_row['name']) &
            (old_df['date'] == new_row['date']) &
            (old_df['session'] == new_row['session']) &
            (old_df['numbers_status'] == new_row['numbers_status'])
        )
        match = old_df[mask]

        if len(match) == 0:
            print(f"[신규 추가] {new_row['name']} - {new_row['date']} {new_row['session']}")
            updated_rows.append(new_row)
        else:
            old_row = match.iloc[0]
            if str(new_row['reservated_num']) != str(old_row['reservated_num']):
                print(f"[업데이트] {new_row['name']} - {new_row['date']} {new_row['session']} 인원 변경됨")
                old_df.drop(index=match.index, inplace=True)
                updated_rows.append(new_row)

    # 병합 후 저장
    updated_df = pd.concat([old_df, pd.DataFrame(updated_rows)], ignore_index=True)
    updated_df.to_csv("./예약현황/서울형_키즈카페_예약현황.csv", index=False, encoding='utf-8')
    print(f"\n 저장 완료! 추가/업데이트된 행: {len(updated_rows)}개")

In [6]:
df = pd.read_csv('./예약현황/서울형_키즈카페_예약현황.csv', encoding='utf-8')

In [8]:
df.drop_duplicates(subset=['name', 'date', 'session', 'numbers_status'], keep='last')

,name,date,reservation_status,capacity_person,capacity_people,session,numbers_status,reservated_num
0,서울형 키즈카페 시립 1호점,4월1,예약가능,33,33,1회,공용,0
1,서울형 키즈카페 시립 1호점,4월1,예약가능,33,33,2회,공용,2
2,서울형 키즈카페 시립 1호점,4월1,예약가능,33,33,3회,공용,2
3,서울형 키즈카페 시립 1호점,4월2,예약가능,33,33,1회,공용,0
4,서울형 키즈카페 시립 1호점,4월2,예약가능,33,33,2회,공용,2
...,...,...,...,...,...,...,...,...
9320,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),4월20,예약가능,35,50,1회,개인,16
9321,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),4월20,예약가능,35,50,2회,개인,22
9322,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),4월24,예약가능,35,50,3회,개인,22
9323,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),4월26,예약가능,35,50,1회,개인,15


In [10]:
df = df.sort_values(['name', 'date', 'session'])

In [12]:
df.to_csv('./예약현황/10일까지_중복행제거_예약현황.csv', index=False, encoding='utf-8')

In [14]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("./예약현황/10일까지_중복행제거_예약현황.csv", encoding="utf-8")

# 기준이 되는 컬럼들 (중복 판단 기준)
subset_cols = ['name', 'date', 'reservation_status', 'capacity_person', 'capacity_people', 'session']

# 해당 기준으로 마지막 행만 남김
df_dedup = df.drop_duplicates(subset=subset_cols, keep='last')

# 결과 저장 (선택)
df_dedup.to_csv("./예약현황/예약현황_중복제거.csv", index=False, encoding="utf-8-sig")

print(f"처리 전: {len(df)}행 → 처리 후: {len(df_dedup)}행")

처리 전: 9325행 → 처리 후: 5672행


In [ ]:
import pandas as pd

# 1. 파일 불러오기
df = pd.read_csv("./예약현황/예약현황_중복제거.csv", encoding="utf-8")

# 2. '4월1' → '4월 1일' 변환
df['date_clean'] = df['date'].str.replace(r'(\d+)월(\d+)', r'\1월 \2일', regex=True)

# 3. 날짜 datetime 변환
df['date_parsed'] = pd.to_datetime("2025 " + df['date_clean'], format="%Y %m월 %d일")

# 4. 정렬: 날짜 → name → session 기준
df_sorted = df.sort_values(by=['name', 'date_parsed', 'session']).reset_index(drop=True)

# 5. 중간 컬럼 제거
df_sorted.drop(columns=['date_clean', 'date_parsed'], inplace=True)

# 6. 저장
df_sorted.to_csv("./예약현황/예약현황_정렬완료.csv", index=False, encoding="utf-8")

print("저장")

In [18]:
import pandas as pd

# 파일 로드
df = pd.read_csv('./예약현황/예약현황_정렬완료.csv')

# 컬럼 정리 (혹시 컬럼명에 공백 있으면 제거)
df.columns = df.columns.str.strip()

# 숫자형 변환 (오류 무시)
df['capacity_person'] = pd.to_numeric(df['capacity_person'], errors='coerce')
df['capacity_people'] = pd.to_numeric(df['capacity_people'], errors='coerce')
df['reservated_num'] = pd.to_numeric(df['reservated_num'], errors='coerce')
df['numbers_status'] = df['numbers_status'].astype(str).str.strip()  # <- 여기 중요!

# 보간 조건: '공용' + 음수
cond = (df['numbers_status'] == '공용') & (df['reservated_num'] < 0)

# 보간 계산
df.loc[cond, 'reservated_num'] = (
    (df.loc[cond, 'capacity_person'] - df.loc[cond, 'capacity_people']).abs()
    + df.loc[cond, 'reservated_num'].abs()
)

# 여전히 음수가 있는지 확인
remaining_negatives = df[(df['numbers_status'] == '공용') & (df['reservated_num'] < 0)]

print("보간 후에도 여전히 남아있는 음수 개수:", len(remaining_negatives))
print(remaining_negatives[['capacity_person', 'capacity_people', 'reservated_num']])

# 저장
df.to_csv('./예약현황/예약현황_보간완료.csv', index=False, encoding='utf-8-sig')

보간 후에도 여전히 남아있는 음수 개수: 0
Empty DataFrame
Columns: [capacity_person, capacity_people, reservated_num]
Index: []


,name,date,reservation_status,capacity_person,capacity_people,session,numbers_status,reservated_num
